In [1]:
import pandas as pd
import warnings
import numpy as np

warnings.filterwarnings('ignore')

raw_df=pd.read_csv('dataset.csv').drop_duplicates()

df=raw_df.copy()

df=df.loc[:,df.columns.drop('FlownYear')] #FlownYear not really relevant

df=df[df['Revenue']>0]

# Get price column (target regression variable)
df['PricePerWeight']=df['Revenue']/df['ChargeableWeight']

### 1.1 Data pre-processing

We will now perform a transformation of categorical variables to obtain a numeric representation that the model is able to understand. For the numeric variables, we will remove outliers, and perform log transformation, as in the data analysis phase we noted they have a highly left-skewed distribution, and many outliers present as well.

In [2]:
products_to_undersample=['DLG','MGK']

for product in products_to_undersample:

    total_samples=df[df['ProductCode']==product].shape[0]

    random_indexes = df.sample(n=total_samples//2, random_state=42).index

    df=df.drop(index=random_indexes)

df.reset_index(inplace=True,drop=True)

In [3]:
from sklearn.preprocessing import LabelEncoder

######################################################################
#                                                                    #
#                      numeric columns                               #
#                                                                    #    
######################################################################
numeric_columns = df.select_dtypes(include=['number']).columns

for column in ['PricePerWeight','Revenue']:

    df[f'Log{column}'] = np.log1p(df[column])  # perform log transformation to scale values within smaller range.

    q_hi  = df[f"Log{column}"].quantile(0.95)

    df = df[(df[f"Log{column}"] < q_hi)] # remove top 5% data points, considered outliers.

    df=df.loc[:,df.columns.drop(column)]
    

######################################################################
#                                                                    #
#                      categorical columns                           #
#                                                                    #    
######################################################################



categorical_columns = df.select_dtypes(include=['category', 'object']).columns

labels_dict={}

for variable in categorical_columns:

    label_encoder = LabelEncoder()
    
    df[f'{variable}_encoded'] = label_encoder.fit_transform(df[variable])

    labels_dict[variable]=label_encoder.classes_

df=df.loc[:,df.columns.drop(categorical_columns)]

Since we want to stratify the train and test parition by product code, to keep the same products distribution in both sets, we will remove products with very low frequencies in order to be able to properly make train and test split.

In [4]:
from sklearn.model_selection import train_test_split                                                                                                               

shipment_frequency=df['ProductCode_encoded'].value_counts().reset_index()

products_to_keep=shipment_frequency[shipment_frequency['count']>3]['ProductCode_encoded'].values

df=df[df['ProductCode_encoded'].isin(products_to_keep)]

df.reset_index(inplace=True)

X_train, X_test, y_train, y_test = train_test_split(
    df.loc[:,df.columns.drop(['LogRevenue', 'LogPricePerWeight'])], df.loc[:,'LogPricePerWeight'], test_size=0.30, random_state=42, stratify=df.loc[:,'ProductCode_encoded'])

### 1.2 Model train

For the model trainning phase, we tried 3 different tree-based models to be able to handle the skewed data distributions. We started from the least complex one: Random Forest, and increased the complexity to XGBoost and LightGBM based on the experimental results obtained.

Each model was evaluated in the following manner:

Having as initial performance metric the RMSE, to have an interpretable metric in terms of dollars, as well as the MAPE, to measure on average, in which percentage the model is mistaken.

    1. 10-folds cross-validation was performed on the train data to asses model's capablity of generalization independent of the data split. The desired output is an average RMSE one across folds, as well as an standard deviation close to zero.

    2. Performance on the test set. After assesing cross-validation. Each model is fitted with the full trainning data, and evaluated on a dataset it has never seen (the test set).

    3. Further analysis is carried on the final test set performance, to asses that the model performs well for all samples, and not only a subset of them. For this reason, test performance by product code is analyzed. Where the ideal scenario is to have a consistent level of performance for all products.

### Random Forest

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# Calculate the sample weights based on the frequency of each product
product_counts = df['ProductCode_encoded'].value_counts()
sample_weights = df['ProductCode_encoded'].map(lambda x: 1 / product_counts[x])

kf = KFold(n_splits=10, shuffle=True, random_state=42)
rmse_scores = []
mape_scores = []

for train_idx, val_idx in kf.split(X_train):
    # Split the data
    X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
    weights_fold_train = sample_weights.iloc[train_idx]

    # Train the model
    rf_regr = RandomForestRegressor(random_state=42)
    rf_regr.fit(X_fold_train, y_fold_train, sample_weight=weights_fold_train)

    # Validate and calculate RMSE and MAPE score
    y_pred = rf_regr.predict(X_fold_val)

    y_pred = np.exp(y_pred)  # Exponentiate to original scale
    
    y_true = np.exp(y_fold_val)  # Exponentiate true values to original scale

    # RMSE Calculation
    fold_rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    rmse_scores.append(fold_rmse.item())

    # MAPE Calculation
    fold_mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100  # MAPE as percentage
    mape_scores.append(fold_mape.item())

# Calculate mean and standard deviation of RMSE and MAPE scores
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

mean_mape = np.mean(mape_scores)
std_mape = np.std(mape_scores)

# Print results for RMSE and MAPE
print(f"RMSE scores for each fold: {rmse_scores}")
print(f"Mean RMSE score: {mean_rmse}")
print(f"Standard deviation of RMSE score: {std_rmse}")
print()
print(f"MAPE scores for each fold: {mape_scores}")
print(f"Mean MAPE score: {mean_mape}")
print(f"Standard deviation of MAPE score: {std_mape}")


RMSE scores for each fold: [2.0897202508533796, 2.7787296315308376, 2.549284685620141, 4.832698613007814, 2.641506662108192, 4.557960792042662, 1.8103624132490945, 4.163184289197137, 2.7236438410392307, 4.759896776311086]
Mean RMSE score: 3.2906987954959575
Standard deviation of RMSE score: 1.099638321110377

MAPE scores for each fold: [10.704045903058695, 9.1090169792895, 6.933903773901018, 10.152395683914035, 8.23210088286394, 10.529550067156455, 6.908186112881628, 8.858179593864069, 8.307249986720024, 9.714290752088566]
Mean MAPE score: 8.944891973573794
Standard deviation of MAPE score: 1.29603866882651


In [6]:
from utils import evaluate_model

# Calculate the sample weights based on the frequency of each product
product_counts = X_train['ProductCode_encoded'].value_counts()
sample_weights = X_train['ProductCode_encoded'].map(lambda x: 1 / product_counts[x])

rf_regr.fit(X_train, y_train, sample_weight=sample_weights)

evaluate_model(rf_regr, X_train, y_train, X_test, y_test, products_to_keep, labels_dict)


Test set R^2: 0.7147433520467911
Test set MSE: 12.163449194520501
Test set MAPE: 8.801522648206465


Summary statistics for errors (all samples):
              RMSE         MAPE
count  1583.000000  1583.000000
mean      0.723822     8.801523
std       3.412754    20.566374
min       0.000000     0.000000
25%       0.005879     0.372063
50%       0.034882     1.912161
75%       0.201689     7.608858
max      47.188086   335.376794





Performance metrics for each product:
   Product Name       R^2       RMSE       MAPE
0           DLJ  0.688020   4.318826  10.745528
1           MGK  0.651928   1.464979   3.064392
2             X  0.968777   0.583338   4.868692
3           ZXM  0.991764   0.386721   3.203877
4           XGS  0.874418   2.179742  18.766873
5           XGK  0.717380   0.691074  12.036164
6             Y -2.222540   3.750093  40.658744
7           SJD  0.958989   1.831735  11.818903
8           PEU  0.725610   2.170547  10.488110
9           XGV  0.172573   2.257166  17.695087
10          RLT  0.953523   0.239394   8.159590
11          MGV -2.041544   0.869258  27.915007
12          XGT  0.517965  13.697617  24.308974

Low performance products (R^2 < 0.8):
   Product Name       R^2       RMSE       MAPE
0           DLJ  0.688020   4.318826  10.745528
1           MGK  0.651928   1.464979   3.064392
5           XGK  0.717380   0.691074  12.036164
6             Y -2.222540   3.750093  40.658744
8         

### XGBoost

In [ ]:
from xgboost import XGBRegressor

# Initialize the XGBRegressor
xgb_regr = XGBRegressor(random_state=0, eval_metric='rmse')

# Perform 10-fold cross-validation
cv_scores = cross_val_score(xgb_regr, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')  # RMSE as the metric

# Print results
print("Cross-validation RMSE scores:", -cv_scores)  # Convert to positive RMSE values
print()
print("Mean RMSE score:", -np.mean(cv_scores))  # Mean RMSE
print("Standard deviation of RMSE scores:", np.std(cv_scores))  # Standard deviation of RMSE


Cross-validation RMSE scores: [0.15581488 0.239099   0.28996469 0.2264444  0.27551143 0.17803577
 0.19154593 0.18785071 0.1745908  0.22538566]

Mean RMSE score: 0.21442432736234993
Standard deviation of RMSE scores: 0.042292949172562065


In [ ]:
# Define custom MAPE scoring function
def mape_score(y_true, y_pred):
    """Computes the Mean Absolute Percentage Error"""
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Initialize the XGBRegressor
xgb_regr = XGBRegressor(random_state=0, eval_metric='rmse')

# Perform 10-fold cross-validation for RMSE
cv_rmse_scores = cross_val_score(xgb_regr, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')  # RMSE as the metric

# Perform 10-fold cross-validation for MAPE
def custom_mape_scorer(model, X, y):
    """Compute the MAPE using the custom scoring function"""
    y_pred = model.predict(X)
    y_pred_exp = np.exp(y_pred)  # Exponentiate predictions back to original scale
    y_true_exp = np.exp(y)  # Exponentiate true values to original scale
    return mape_score(y_true_exp, y_pred_exp)

cv_mape_scores = cross_val_score(xgb_regr, X_train, y_train, cv=10, scoring=custom_mape_scorer)  # MAPE as the custom metric

# Print RMSE results
print("Cross-validation RMSE scores:", -cv_rmse_scores)  # Convert to positive RMSE values
print()
print("Mean RMSE score:", -np.mean(cv_rmse_scores))  # Mean RMSE
print()
print("Standard deviation of RMSE scores:", np.std(cv_rmse_scores))  # Standard deviation of RMSE

print()
print()
# Print MAPE results
print("Cross-validation MAPE scores:", cv_mape_scores)
print()
print("Mean MAPE score:", np.mean(cv_mape_scores))
print()
print("Standard deviation of MAPE scores:", np.std(cv_mape_scores))


Cross-validation RMSE scores: [0.15581488 0.239099   0.28996469 0.2264444  0.27551143 0.17803577
 0.19154593 0.18785071 0.1745908  0.22538566]

Mean RMSE score: 0.21442432736234993

Standard deviation of RMSE scores: 0.042292949172562065


Cross-validation MAPE scores: [ 7.58276993  7.93145573  8.22855011  8.76017115  8.518469    7.57306015
 10.10924373  8.5694164   9.43449525  8.3926628 ]

Mean MAPE score: 8.510029425089915

Standard deviation of MAPE scores: 0.7512500928878331


In [9]:
xgb_regr.fit(X_train,y_train)

evaluate_model(xgb_regr, X_train, y_train, X_test, y_test, products_to_keep, labels_dict)


Test set R^2: 0.7959035294483118
Test set MSE: 8.702749149403244
Test set MAPE: 8.663532100195367


Summary statistics for errors (all samples):
              RMSE         MAPE
count  1583.000000  1583.000000
mean      0.654460     8.663532
std       2.877440    21.105899
min       0.000026     0.001709
25%       0.014282     0.878355
50%       0.049782     2.673703
75%       0.219171     8.389122
max      42.649660   365.508747





Performance metrics for each product:
   Product Name       R^2       RMSE       MAPE
0           DLJ  0.789562   3.547032  10.033997
1           MGK  0.687073   1.389052   3.044204
2             X  0.963576   0.630051   7.041296
3           ZXM  0.973743   0.690492   4.549631
4           XGS  0.876714   2.159727  20.371264
5           XGK  0.760365   0.636354   9.764293
6             Y -3.623274   4.491773  43.814547
7           SJD  0.888127   3.025335  14.653437
8           PEU  0.396299   3.219555  17.438340
9           XGV -0.270863   2.797357  18.548332
10          RLT  0.968824   0.196065   7.002168
11          MGV -0.715160   0.652760  17.389493
12          XGT  0.527947  13.555039  34.781658

Low performance products (R^2 < 0.8):
   Product Name       R^2       RMSE       MAPE
0           DLJ  0.789562   3.547032  10.033997
1           MGK  0.687073   1.389052   3.044204
5           XGK  0.760365   0.636354   9.764293
6             Y -3.623274   4.491773  43.814547
8         

### LightGBM

In [10]:
import lightgbm as lgb

# Define custom MAPE function
def mape_score(y_true, y_pred):
    """Computes the Mean Absolute Percentage Error"""
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Initialize K-Fold Cross Validation with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# LightGBM parameters
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'max_depth': 10,
    'num_leaves': 31,
    'random_state': 42,
    'verbose': -1,
    'is_unbalance': True,   # Enable imbalance handling
    'min_child_weight': 1.5,  # Prevent overfitting
    'min_data_in_leaf': 20    # Prevent overfitting
}

# Store fold performances
fold_rmse_scores = []
fold_mape_scores = []

# Cross-validation loop
for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
    # Split data into training and validation for this fold
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Prepare LightGBM datasets
    train_data = lgb.Dataset(X_train_fold, label=y_train_fold)
    val_data = lgb.Dataset(X_val_fold, label=y_val_fold, reference=train_data)

    # Train LightGBM model
    lgbm_model = lgb.train(
        params,
        train_data,
        valid_sets=[train_data, val_data],
        num_boost_round=1000
    )

    # Predict and evaluate on the validation set
    y_val_pred = lgbm_model.predict(X_val_fold)
    
    # Exponentiate predictions and true values to return to original scale
    y_val_pred_exp = np.exp(y_val_pred)
    y_val_exp = np.exp(y_val_fold)

    # Calculate RMSE for this fold (on original scale)
    fold_rmse = np.sqrt(mean_squared_error(y_val_exp, y_val_pred_exp))
    fold_rmse_scores.append(fold_rmse.item())

    # Calculate MAPE for this fold (on original scale)
    fold_mape = mape_score(y_val_exp, y_val_pred_exp)
    fold_mape_scores.append(fold_mape.item())

    # Optionally print fold-wise results
    # print(f"Fold {fold + 1} RMSE: {fold_rmse}, MAPE: {fold_mape}")

# Calculate mean and standard deviation of RMSE and MAPE scores
mean_rmse = np.mean(fold_rmse_scores)
std_rmse = np.std(fold_rmse_scores)
mean_mape = np.mean(fold_mape_scores)
std_mape = np.std(fold_mape_scores)

# Print results

print("\n10-Fold Cross-Validation Results:")
print()
print(fold_rmse_scores)
print(f"Mean RMSE: {mean_rmse}")
print(f"Standard Deviation of RMSE: {std_rmse}")
print()
print(fold_mape_scores)
print(f"Mean MAPE: {mean_mape}")
print(f"Standard Deviation of MAPE: {std_mape}")



10-Fold Cross-Validation Results:

[1.4176870490472415, 2.2386550354502934, 2.6199560975501166, 4.055175951456072, 1.9508788825714865, 4.41513530896753, 1.7277917389890256, 3.3531470759046273, 2.577460742703032, 4.0740593486300964]
Mean RMSE: 2.842994723126952
Standard Deviation of RMSE: 1.013573687657744

[11.117150682299432, 9.517445703672278, 6.800089799343431, 8.759486519360724, 7.411048955255605, 10.21756788818108, 8.117357543262246, 8.706301784197723, 7.377215343640085, 9.327215000393858]
Mean MAPE: 8.735087921960647
Standard Deviation of MAPE: 1.286809917312812


In [11]:
train_data = lgb.Dataset(X_train, label=y_train)

# Train LightGBM model on the full training data
lgbm_model = lgb.train(
    params,
    train_data,
    num_boost_round=1000
)

evaluate_model(lgbm_model, X_train, y_train, X_test, y_test, products_to_keep, labels_dict)

Test set R^2: 0.8254922579943109
Test set MSE: 7.441074797614767
Test set MAPE: 8.179852404964171


Summary statistics for errors (all samples):
              RMSE         MAPE
count  1583.000000  1583.000000
mean      0.592721     8.179852
std       2.663501    18.192174
min       0.000019     0.001347
25%       0.014676     0.945674
50%       0.051475     2.665119
75%       0.218651     8.000207
max      40.686308   330.546873





Performance metrics for each product:
   Product Name       R^2      RMSE       MAPE
0           DLJ  0.805678  3.408506   9.709693
1           MGK  0.888326  0.829798   2.228377
2             X  0.918645  0.941616   6.311620
3           ZXM  0.987780  0.471067   5.199593
4           XGS  0.961628  1.204891  14.167158
5           XGK  0.765753  0.629159  11.083232
6             Y -2.668451  4.001145  43.078690
7           SJD  0.938969  2.234536  12.989922
8           PEU  0.687921  2.314822  15.534776
9           XGV -0.450582  2.988614  33.336571
10          RLT  0.751877  0.553130  10.608330
11          MGV -0.951417  0.696268  23.080060
12          XGT  0.884302  6.710708  17.770797

Low performance products (R^2 < 0.8):
   Product Name       R^2      RMSE       MAPE
5           XGK  0.765753  0.629159  11.083232
6             Y -2.668451  4.001145  43.078690
8           PEU  0.687921  2.314822  15.534776
9           XGV -0.450582  2.988614  33.336571
10          RLT  0.751877  0.

## 1.3 Conclusions

We have chosen **RMSE (Root Mean Squared Error)** as our primary evaluation metric because it provides an interpretable measure of monetary loss. RMSE directly quantifies the magnitude of prediction errors, offering a clear understanding of how much money is being lost due to incorrect predictions. For example, an RMSE of 2.73 means that, on average, our predictions deviate by about 2.73 units. This is critical in a financial context, as even small deviations can lead to significant losses. In our case, with a **mean chargeable weight** of around **2276.93 units**, an error of 2.73 represents a significant financial impact. Additionally, we have prioritized **MAPE (Mean Absolute Percentage Error)** because it allows us to assess the model's accuracy in percentage terms. This is important for evaluating the relative error across different product categories, regardless of their size or value.

### Conclusion

Based on the results, **LightGBM** is the top-performing model in terms of both **R²** and **RMSE**. Its average test set performance, with an R² of 0.83 and RMSE of 2.73, outperforms both Random Forest and XGBoost, indicating that it generalizes better across the data. Furthermore, its MAPE is competitive with the other models, meaning its percentage error is on par with the others, making it a strong choice overall.

However, even though **LightGBM** performs well, the **RMSE** value of 2.73 still indicates room for improvement. Given that the **mean chargeable weight** is around **2276.93 units**, this error can have a significant impact on financial outcomes, meaning that there is still considerable modeling work to do. The RMSE suggests that our predictions are still deviating enough to result in non-negligible monetary loss. This points to the fact that the model has not fully captured the underlying patterns in the data, and we have room to improve the prediction accuracy, especially for products with complex or challenging characteristics.

### Factors Affecting Model Performance

Several key factors contribute to the inability to achieve better performance:

1. **Outliers**:
   The data contains significant outliers, particularly in the performance of products like **Y, XGT,** and **MGV**, which show very low R² values. Outliers represent extreme cases where the actual data points deviate significantly from the general trend. In such cases, the model might struggle to generalize well, as these extreme values can disproportionately influence the results. The current models, while robust to some degree, are not entirely immune to the effect of outliers. They tend to fit too closely to these anomalies, leading to poor generalization across the rest of the data.

2. **Skewed Data**:
   Another challenge comes from the skewed nature of the data. If certain products or categories are underrepresented, the models may develop a bias toward the majority classes, making it difficult to capture the unique characteristics of minority classes. This skewed distribution can lead to inaccurate predictions for underrepresented products. For instance, if a particular product category is rare, the model may not learn enough about its behavior, resulting in larger errors when making predictions for those specific products.

3. **Heterogeneity Across Products**:
   The diverse nature of the products also makes it difficult to achieve consistent performance. Products vary in terms of their characteristics, behaviors, and pricing structures, leading to different levels of prediction accuracy. Some products may follow predictable patterns that models like LightGBM and XGBoost can capture well, while others may have more complex relationships that are harder to model. As a result, the performance for certain products remains suboptimal.

### Plan to Improve the Modelling Process

1. **Hyperparameter Tuning**:  
   Hyperparameter tuning can significantly enhance the performance of all models. For **LightGBM**, tuning parameters such as `num_leaves`, `max_depth`, and `min_data_in_leaf` could help reduce overfitting and further improve model generalization.  
   For **XGBoost**, focusing on parameters like `learning_rate`, `max_depth`, and `subsample` can optimize predictive accuracy.  
   For **Random Forest**, adjusting the `n_estimators` and `max_depth` hyperparameters can help stabilize performance and make it more consistent.

2. **Oversampling the Data**:  
    As the dataset is highly imbalanced in terms of product code oversampling would prevent the model from being biased toward the majority class. It would ensure a more balanced dataset, allowing for more accurate and a model with more generalization power.

3. **Clustering for Grouping Products**:  
   Given the variation in performance across different products, it may be beneficial to apply **clustering** techniques to group similar products together and train separate models for each product group. This can help improve model performance by addressing specific characteristics unique to different product categories. This can lead to more tailored predictions and potentially reduce error rates for products that are more challenging to predict.

4. **Training Neural Networks**:  
   Given the potential for further model improvement, we also plan to explore **neural networks**. Neural networks are capable of capturing complex patterns and relationships in the data that may not be easily detected by traditional tree-based models like LightGBM, XGBoost, and Random Forest. By training neural networks, we can further reduce prediction errors and improve overall model accuracy, especially for products with more complex behaviors or patterns.


By focusing on these areas—hyperparameter tuning, oversampling, clustering, and neural network training, we can make significant improvements to model performance. Despite the progress made so far, there's still room for refinement to reduce errors and enhance the predictive accuracy of our models. These improvements will ensure more reliable predictions, particularly for products with complex or variable characteristics.